In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load dataset
data = pd.read_csv('chatbot_data.csv')

# Extract questions and responses
questions = data['question'].tolist()
responses = data['response'].tolist()

# Tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(questions + responses)

vocab_size = len(tokenizer.word_index) + 1
max_seq_length = max(max(len(q.split()) for q in questions), max(len(r.split()) for r in responses))

def tokenize_and_pad(texts):
    sequences = tokenizer.texts_to_sequences(texts)
    return pad_sequences(sequences, maxlen=max_seq_length, padding='post')

input_sequences = tokenize_and_pad(questions)
target_sequences = tokenize_and_pad(responses)

# Prepare decoder input and output sequences
decoder_input_sequences = np.zeros_like(target_sequences)
decoder_input_sequences[:, 1:] = target_sequences[:, :-1]
decoder_output_sequences = np.expand_dims(target_sequences, -1)

# Define model architecture
def create_seq2seq_model(vocab_size, max_seq_length):
    # Encoder
    encoder_inputs = Input(shape=(max_seq_length,))
    encoder_embedding = Embedding(vocab_size, 128)(encoder_inputs)
    _, state_h, state_c = LSTM(128, return_state=True)(encoder_embedding)
    encoder_states = [state_h, state_c]

    # Decoder
    decoder_inputs = Input(shape=(max_seq_length,))
    decoder_embedding = Embedding(vocab_size, 128)(decoder_inputs)
    decoder_lstm = LSTM(128, return_sequences=True)(decoder_embedding, initial_state=encoder_states)
    decoder_outputs = Dense(vocab_size, activation='softmax')(decoder_lstm)

    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

model = create_seq2seq_model(vocab_size, max_seq_length)

# Train the model
model.fit([input_sequences, decoder_input_sequences], decoder_output_sequences,
          epochs=2000, batch_size=2, validation_split=0.1)

# Save the model
model.save('chatbot_model.h5')
print("Model saved as chatbot_model.h5")


Epoch 1/2000
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.4751 - loss: 4.8678 - val_accuracy: 0.5417 - val_loss: 2.8884
Epoch 2/2000
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5686 - loss: 2.4771 - val_accuracy: 0.5521 - val_loss: 2.8477
Epoch 3/2000
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5885 - loss: 2.2572 - val_accuracy: 0.5521 - val_loss: 2.9000
Epoch 4/2000
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5838 - loss: 2.1995 - val_accuracy: 0.5521 - val_loss: 2.9139
Epoch 5/2000
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6352 - loss: 1.9396 - val_accuracy: 0.5521 - val_loss: 2.9366
Epoch 6/2000
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6146 - loss: 1.9440 - val_accuracy: 0.5521 - val_loss: 2.9639
Epoch 7/2000
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6375 - loss: 1.9092 - val_accuracy: 0.5521 - val_loss: 2.9963
Epoch 8/2000
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6404 - loss: 1.7828 - val_accuracy: 0

Model saved as chatbot_model.h5


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the tokenizer (make sure it's the same one used during training)
tokenizer = Tokenizer()

# Correct max sequence length (set to 16 based on the error)
max_seq_length = 16

# Load the trained model
model = tf.keras.models.load_model('chatbot_model.h5')

# Function to preprocess the input
def preprocess_input(text, tokenizer, max_seq_length):
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_seq_length, padding='post')
    return padded_sequence

# Function to generate response from the model
def generate_response(input_text, model, tokenizer, max_seq_length):
    preprocessed_input = preprocess_input(input_text, tokenizer, max_seq_length)
    
    # Start token sequence (initialize with zeros or specific start token)
    decoder_input_sequence = np.zeros((1, max_seq_length))
    
    response = []
    for _ in range(max_seq_length):
        # Predict the next token
        predicted_sequence = model.predict([preprocessed_input, decoder_input_sequence])
        
        # Get the most probable word index
        predicted_index = np.argmax(predicted_sequence[0], axis=-1)[0]
        
        # Add predicted word to the response
        response.append(predicted_index)
        
        # Update the decoder input with the new word
        decoder_input_sequence[0] = np.roll(decoder_input_sequence[0], -1)
        decoder_input_sequence[0, -1] = predicted_index
        
        # Break if the end token is reached
        if predicted_index == tokenizer.word_index['<end>']:
            break
    
    # Convert indices to words
    response_words = [tokenizer.index_word.get(idx, '') for idx in response]
    
    return ' '.join(response_words).strip()



In [4]:
import pygame
import random
import sys

# Initialize Pygame
pygame.init()

# Screen dimensions
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
FPS = 60

# Colors
WHITE = (255, 255, 255)
RED = (255, 0, 0)

# Note class
class Note:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.speed = 5

    def update(self):
        self.y += self.speed

    def draw(self, screen):
        pygame.draw.rect(screen, RED, (self.x, self.y, 50, 50))

# Set up display
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption('Rhythm Game')

# Clock for controlling frame rate
clock = pygame.time.Clock()

# List to hold notes
notes = []

# Game loop
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    # Clear the screen
    screen.fill(WHITE)

    # Update and draw notes
    for note in notes:
        note.update()
        note.draw(screen)
        if note.y > SCREEN_HEIGHT:
            notes.remove(note)

    # Generate new notes
    if random.randint(1, 20) == 1:
        notes.append(Note(random.randint(0, SCREEN_WIDTH - 50), -50))

    # Check for key presses
    keys = pygame.key.get_pressed()
    if keys[pygame.K_a]:  # Example for 'A' key
        for note in notes:
            if note.y > SCREEN_HEIGHT - 100 and note.x < 100 and note.x + 50 > 0:
                notes.remove(note)
                break

    # Update display
    pygame.display.flip()

    # Cap the frame rate
    clock.tick(FPS)


pygame 2.6.0 (SDL 2.28.4, Python 3.12.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
import pickle

# Save the tokenizer after training
with open('tokenizer.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Load the tokenizer before testing
with open('tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)


In [1]:
preprocessed_input = preprocess_input(input_text, tokenizer, max_seq_length)
print("Tokenized and Padded Input:", preprocessed_input)


NameError: name 'preprocess_input' is not defined

In [2]:
predicted_sequence = model.predict([preprocessed_input, decoder_input_sequence])
print("Predicted Sequence:", predicted_sequence)


NameError: name 'model' is not defined

In [5]:
import pygame
import sys
import random

# Initialize Pygame
pygame.init()

# Constants
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
FPS = 60
NOTE_SPEED = 5
NOTE_WIDTH = 60
NOTE_HEIGHT = 60

# Colors
WHITE = (255, 255, 255)
RED = (255, 0, 0)

# Screen setup
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption('Rhythm Game')

# Clock
clock = pygame.time.Clock()

# Load music
pygame.mixer.music.load('REEEEEEE.ogg')  # Load OGG music file
pygame.mixer.music.play()

# Note class
class Note:
    def __init__(self, x):
        self.x = x
        self.y = -NOTE_HEIGHT

    def update(self):
        self.y += NOTE_SPEED

    def draw(self, screen):
        pygame.draw.rect(screen, RED, (self.x, self.y, NOTE_WIDTH, NOTE_HEIGHT))

# Generate notes
def generate_notes():
    return [Note(random.randint(0, SCREEN_WIDTH - NOTE_WIDTH)) for _ in range(5)]

# Game loop
notes = generate_notes()
score = 0
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    # Clear screen
    screen.fill(WHITE)

    # Update notes
    for note in notes:
        note.update()
        note.draw(screen)
        if note.y > SCREEN_HEIGHT:
            notes.remove(note)

    # Input handling
    keys = pygame.key.get_pressed()
    if keys[pygame.K_a]:  # Example for 'A' key
        for note in notes:
            if note.y > SCREEN_HEIGHT - 100 and note.x < 100 and note.x + NOTE_WIDTH > 0:
                notes.remove(note)
                score += 1
                break

    # Generate new notes
    if random.randint(1, 60) == 1:
        notes.append(Note(random.randint(0, SCREEN_WIDTH - NOTE_WIDTH)))

    # Display score
    font = pygame.font.SysFont(None, 36)
    text = font.render(f'Score: {score}', True, (0, 0, 0))
    screen.blit(text, (10, 10))

    # Update display
    pygame.display.flip()
    clock.tick(FPS)


SystemExit: 

In [1]:
import os

class CTBInterpreter:
    def __init__(self, modules_dir="ctb_modules"):
        self.modules_dir = modules_dir
        if not os.path.exists(self.modules_dir):
            os.makedirs(self.modules_dir)
        self.package_code = []  # Store package code lines

    def eval(self, code):
        lines = code.split('\n')
        for line in lines:
            self.execute(line.strip())

    def execute(self, line):
        if line.startswith("let "):  # Handle variable declaration
            self.package_code.append(line)  # Store the code
        elif line.startswith("deck shuff"):  # Handle deck shuff command
            self.handle_deck_shuff(line)
        else:
            self.package_code.append(line)  # Store the code

    def handle_deck_shuff(self, line):
        # Extract package name
        parts = line.split('/')
        if len(parts) == 2:
            package_type, package_name = parts[0].split()[2], parts[1]
            self.create_dct_file(package_name)
        else:
            print("Invalid deck shuff command.")

    def create_dct_file(self, package_name):
        dct_file_path = os.path.join(self.modules_dir, f"{package_name}.dct")
        with open(dct_file_path, 'w') as dct_file:
            dct_file.write('\n'.join(self.package_code))
        print(f"Package '{package_name}' has been converted to a .dct file at {dct_file_path}.")

# Example usage
ctb_code = """
let x = 10
let y = 20
deck shuff ex/mypackage
"""

interpreter = CTBInterpreter()
interpreter.eval(ctb_code)


Package 'mypackage' has been converted to a .dct file at ctb_modules\mypackage.dct.


In [6]:
import os

# Define a simple interpreter for .ctb files
class CTBInterpreter:
    def __init__(self):
        self.packages = {}

    def dload(self, package_name, code=None):
        if code:
            # Save the package code (simulated here)
            self.packages[package_name] = code
        else:
            # Execute the package code
            if package_name in self.packages:
                exec(self.packages[package_name])
            else:
                print(f"Package {package_name} not found.")

# Simulate package management with deck
class DeckManager:
    def __init__(self):
        self.interpreter = CTBInterpreter()

    def shuff(self, package_path):
        # Read and convert .ctb to .dct (simulated here)
        with open(package_path, 'r') as file:
            code = file.read()
        dct_path = package_path.replace('.ctb', '.dct')
        with open(dct_path, 'w') as file:
            file.write(code)
        print(f"Converted {package_path} to {dct_path}.")

    def load(self, dct_path):
        # Load a .dct package (simulated here)
        package_name = os.path.basename(dct_path).replace('.dct', '')
        with open(dct_path, 'r') as file:
            code = file.read()
        self.interpreter.dload(package_name, code)
        print(f"Loaded {dct_path} into repository.")

    def on(self, package_name):
        # Run a package (simulated here)
        self.interpreter.dload(package_name)

# Example usage
if __name__ == "__main__":
    deck = DeckManager()

    # Convert and load the package
    deck.shuff('hello-world.ctb')
    deck.load('hello-world.dct')

    # Run the package
    deck.on('helloworld')


Converted hello-world.ctb to hello-world.dct.
Loaded hello-world.dct into repository.
Package helloworld not found.


In [1]:
import socket
import threading
import pyautogui
import pickle
import struct

def handle_client(client_socket):
    while True:
        try:
            # Capture screen and send to client
            screenshot = pyautogui.screenshot()
            screenshot_bytes = pickle.dumps(screenshot)
            
            # Send length of data first
            data_length = struct.pack('I', len(screenshot_bytes))
            client_socket.sendall(data_length)
            
            # Send the actual data
            client_socket.sendall(screenshot_bytes)
        except Exception as e:
            print(f"Error: {e}")
            client_socket.close()
            break

def start_server():
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server.bind(('0.0.0.0', 9999))
    server.listen(5)
    print("Server listening on port 9999...")
    
    while True:
        try:
            client_socket, addr = server.accept()
            print(f"Accepted connection from {addr}")
            client_handler = threading.Thread(target=handle_client, args=(client_socket,))
            client_handler.start()
        except Exception as e:
            print(f"Server Error: {e}")

if __name__ == "__main__":
    start_server()


Server listening on port 9999...
Accepted connection from ('127.0.0.1', 59102)
Error: [WinError 10054] An existing connection was forcibly closed by the remote host
Accepted connection from ('127.0.0.1', 59128)
Accepted connection from ('127.0.0.1', 59394)
Error: [WinError 10054] An existing connection was forcibly closed by the remote host
Error: [WinError 10054] An existing connection was forcibly closed by the remote host
